In [ ]:
!pip install python-docx

In [ ]:
from docx import Document
import re

In [ ]:

doc = Document("/kaggle/input/lawlaw2/Luat-To-chuc-VKSND.docx")
answers = []

found_question = False  # Chỉ bắt đầu lấy đáp án sau khi gặp câu hỏi đầu tiên

for para in doc.paragraphs:
    para_text = para.text.strip()

    # Nhận biết câu hỏi đầu tiên xuất hiện
    if not found_question and (para_text.endswith("?") or para_text.startswith("Theo")):
        found_question = True

    if not found_question:
        continue  # Bỏ qua phần đầu chưa chứa câu hỏi

    # Bắt đầu duyệt đáp án đúng
    for run in para.runs:
        text = run.text.strip()
        if not text:
            continue

        has_color = run.font.color and run.font.color.rgb

        if has_color:
            answers.append(text)

# In kết quả
answers = [ans for ans in answers if ans != "?" and ans != ":"]
raw_lines = []
for i, ans in enumerate(answers):
    x= f"{ans}"
    raw_lines.append(x)
    print(f"{ans}")


In [ ]:
import re
import pandas as pd

def extract_qa_from_text(lines):
    questions = []
    current_question = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Nếu bắt đầu bằng "Theo" → là câu hỏi mới
        if line.startswith("Theo"):
            current_question = line
        elif current_question:
            # Dòng sau là đáp án tương ứng
            questions.append({
                "question": current_question,
                "answer": line
            })
            current_question = None  # reset để đón câu hỏi tiếp theo

    return questions

# Giả sử bạn đã có raw_text từ file


qa_pairs = extract_qa_from_text(raw_lines)

# Xuất thành CSV
df = pd.DataFrame(qa_pairs)
df.to_csv("question_answer.csv", index=False, encoding='utf-8-sig')
print(df)


In [ ]:
import pandas as pd

data = []
i = 0
while i < len(answers):
    line = answers[i].strip()

    # Nhận diện câu hỏi mới
    if line.startswith("Theo Luật Luật tổ chức VKSND 2014"):
        question = line
        options = []

        i += 1
        # Lấy các dòng tiếp theo là đáp án cho đến khi gặp câu hỏi tiếp theo hoặc hết
        while i < len(answers) and not answers[i].startswith("Theo Luật Luật tổ chức VKSND 2014"):
            opt = answers[i].strip()
            if opt:
                options.append(opt)
            i += 1

        # Gán các option vào cột phù hợp (A–E), phần còn lại sẽ là None
        row = {"question": question}
        for idx, opt in enumerate(options):
            row[f"option_{chr(65 + idx)}"] = opt  # chr(65) = 'A'
        data.append(row)
    else:
        i += 1

# Chuyển thành DataFrame và lưu CSV
df = pd.DataFrame(data)
df.to_csv("qa2.csv", index=False, encoding="utf-8-sig")
print(df.head())


In [ ]:
import pandas as pd

# Load 2 file
df_options = pd.read_csv("/kaggle/working/qa2.csv")  
df_correct = pd.read_csv("/kaggle/input/qaaaaa/question_answer.csv")  

# Merge theo câu hỏi
df_merged = pd.merge(df_options, df_correct, on="question", how="inner")

# Tạo dataset mới
processed = []

for _, row in df_merged.iterrows():
    correct = row["answer"].strip()

    # Lấy các option A–E (nếu có)
    options = []
    for col in ["option_A", "option_B", "option_C", "option_D"]:
        val = row.get(col)
        if isinstance(val, str) and val.strip():
            options.append(val.strip())

    # Tách các đáp án sai
    incorrects = [opt for opt in options if opt != correct]

    # Tạo dict dữ liệu
    result = {
        "question": row["question"],
        "correct_answer": correct,
    }

    for i, inc in enumerate(incorrects):
        result[f"incorrect_{i+1}"] = inc

    processed.append(result)

# Chuyển thành DataFrame và lưu
df_final = pd.DataFrame(processed).drop(columns=["incorrect_4"])
df_final.to_csv("question_answer_constrast.csv", index=False, encoding="utf-8-sig")
print(df_final.head())
